In [23]:
import SimFunctions
import SimClasses
import SimRNG
import simpy
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy.stats import probplot, kstest
import pickle
from scipy.stats.distributions import norm, expon, erlang, lognorm, weibull_min, gamma
import statsmodels.api as sm
warnings.filterwarnings("ignore")
# fix random number seed
np.random.seed(1)


# Classes

In [130]:
# class Station:
#     def __init__(self, station_id, level, capacity):
#         self.id = station_id
#         self.level = level
#         self.capacity = capacity
        
#         for bike in range(level):
#             self.bike_id = level

#     def rent_bike(self):
#         # Request a bike from the station
#         self.level -= 1

#     def return_bike(self):
#         # Return a bike to the station
#         self.level += 1

# # Initialize the stations and create a dictionary mapping station IDs to Station instances
# Stations = [Station(station_id=1, level=7, capacity=10), Station(station_id=2, level=15, capacity=20)]
# StationDict = {station.id: station for station in Stations}

In [107]:
class Station:
    def __init__(self, station_id, level, capacity):
        self.id = station_id
        self.level = level
        self.capacity = capacity
        self.bikes = []

        for i in range(level):
            bike_id = f"{station_id}-{i+1}"  # create unique bike ID
            self.bikes.append(bike_id)

    def rent_bike(self):
        # Request a bike from the station
        if self.level > 0:
            self.level -= 1
            return self.bikes.pop()
        else:
            return None

    def return_bike(self, bike_id):
        # Return a bike to the station
        if self.level < self.capacity:
            self.level += 1
            self.bikes.append(bike_id)

# Initialize the stations and create a dictionary mapping station IDs to Station instances
Stations = [Station(station_id=1, level=7, capacity=10), Station(station_id=2, level=15, capacity=20)]
StationDict = {station.id: station for station in Stations}

# Functions

In [108]:
ZSimRNG = SimRNG.InitializeRNSeed()
Calendar = SimClasses.EventCalendar()
TheQueues = []
TheCTStats = []
TheDTStats = []
TheResources = []

In [126]:
def Departure(station_id):
    station = StationDict[station_id]
    print("(DEPARTING) Station ID: {} | Level {}".format(station.id, station.level, station.capacity))
    print("-------------------------------------------------------------------------")

    end_id = Destination(station)
    SimFunctions.Schedule(Calendar, "Bike_Arrival", 2*SimRNG.Lognormal(0.1, np.sqrt(0.066), 4))
    
    # Rent a bike from the station
    station.rent_bike()
    return end_id

def Destination(station_id):
    #print("Destination")
    #station = StationDict[station_id]
    
    U = SimRNG.Uniform(0, 1, 3)
    end_id = 2 if U > 1/2 else 1
    
    end_station = StationDict[end_id]
    print("(DESTINATION) Station ID: {} | Level: {}".format(end_station.id, end_station.level))
    print("-------------------------------------------------------------------------")
    return end_id

def Customer_Arrival_Rate(station_id):
    U = SimRNG.Uniform(0, 1, 3)
    station_id = 2 if U > 1/2 else 1
    return station_id
    

def Customer_Arrival(station_id, Empty_Error, bike):
    SimFunctions.Schedule(Calendar, "Customer_Arrival", SimRNG.Expon(0.5, 1))
    station = StationDict[station_id]
    print("(CUSTOMER ARRIVAL) Station ID: {} | Level: {} | Capacity: {}".format(station.id, station.level, station.capacity))
    print("-------------------------------------------------------------------------")
        
    new_station_id = Customer_Arrival_Rate(station_id)

    # Station is empty
    if station.level <= 0:
        print(f"    Customer Arrives at STATION {station.id} -- EMPTY")
        SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(0.1, 5))
        Empty_Error += 1
    else:
        bike = station.rent_bike()
        if bike is None:
            SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(1, 5))
        else:
            SimFunctions.Schedule(Calendar, "Departure", 2*SimRNG.Lognormal(0.1, np.sqrt(0.066), 6))
    return new_station_id, Empty_Error, bike

def Bike_Arrival(end_id, Full_Error, bike):
    destination_station = StationDict[end_id]
    # Destination Station is not full
    if destination_station.level < destination_station.capacity:
        destination_station.return_bike(bike)
        print("(BIKE ARRIVAL) Station ID: {} | Level: {} | Bike ID: {}".format(destination_station.id, destination_station.level, bike))
        print("-------------------------------------------------------------------------")
    else:
    # STATION FULL
        print(f"    Bike Arrives at STATION {destination_station.id} -- FULL")
        SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(0.1, 5))
        Full_Error += 1
    return Full_Error

def Retrial(end_id, Full_Error, bike):
    print("Retrial")
    destination_station = StationDict[end_id]
    # Destination Station is not full
    if destination_station.level < destination_station.capacity:
        destination_station.return_bike(bike)
        print("     Bike has succesfully retried and returned bike to Station", destination_station.id)
    else:
    # STATION FULL
        print(f"    STATION {destination_station.id} FULL")
        SimFunctions.Schedule(Calendar, "Retrial", SimRNG.Expon(0.1, 5))
        Full_Error += 1
    return Full_Error


# Simulation

In [129]:
ZSimRNG = SimRNG.InitializeRNSeed()

Calendar = SimClasses.EventCalendar()

Queue = SimClasses.FIFOQueue()
Wait = SimClasses.DTStat()
Bikes = SimClasses.Resource()

TheCTStats = []
TheDTStats = []
TheQueues = []
TheResources = []

Stations = [7, 15]
Station_Capacity = [10, 20]

Station_1_Resource = []
Station_2_Resource = []

MeanTBA = 0.5
MeanTR = 0.5
MeanST = 0.5

Phases = 3
RunLength = 50
WarmUp = 5
end_id = 1
bike = 0

for reps in range(0, 1, 1):
    Full_Error = 0
    Empty_Error = 0 
    
    inital_station_id = np.random.random_integers(1, 2)
    station_id = inital_station_id
    
    # Initialize the stations and create a dictionary mapping station IDs to Station instances
    Stations = [Station(station_id=1, level=7, capacity=10), Station(station_id=2, level=15, capacity=20)]
    StationDict = {station.id: station for station in Stations}

    SimFunctions.SimFunctionsInit(Calendar, TheQueues, TheCTStats, TheDTStats, TheResources)
    SimFunctions.Schedule(Calendar, "Customer_Arrival", SimRNG.Expon(MeanTBA, 1))
    SimFunctions.Schedule(Calendar, "EndSimulation", RunLength)
    SimFunctions.Schedule(Calendar, "ClearIt", WarmUp)

    NextEvent = Calendar.Remove()
    SimClasses.Clock = NextEvent.EventTime
    if NextEvent.EventType == "Customer_Arrival":
        Customer_Arrival(inital_station_id, Empty_Error, bike)

    for hours in [8, 8.3]:#, 9, 9.3, 10, 10.3, 11, 11.3, 12]:  # 30 min intervals
        print(f"Clock Hour | {hours}")
        for minutes in [i for i in range(1, 31)]:  # 30 min intervals
            print("Minute | {}".format(minutes))
            NextEvent = Calendar.Remove()
            SimClasses.Clock = NextEvent.EventTime

            if NextEvent.EventType == "Customer_Arrival":
                station_id, Empty_Error, bike = Customer_Arrival(station_id, Empty_Error, bike)

            elif NextEvent.EventType == "Departure":
                end_id = Departure(station_id)
            
            elif NextEvent.EventType == "Bike_Arrival":
                Full_Error = Bike_Arrival(end_id, Full_Error, bike)
            
            elif NextEvent.EventType == "Retrial":
                Full_Error = Retrial(end_id, Full_Error, bike)
            
            # station = StationDict[station_id]
            # print("Station ID:", station.id)
            # print("Station Level {} | Capacity {}".format( station.level, station.capacity))
            # print("-------------------------------------------------------------------------")
            # # print("Trip Start ID: {} | End ID: {}".format(trip.id, end_id))
            # print("Station Level {} | Capacity {}".format( station.level, station.capacity))
            
    print(StationDict[1].bikes)
    print(StationDict[2].bikes)
    print("Num of Full Errors:", Full_Error)
    print("Num of Empty Errors:", Empty_Error)

(CUSTOMER ARRIVAL) Station ID: 2 | Level: 15 | Capacity: 20
-------------------------------------------------------------------------
Clock Hour | 8
Minute | 1
(DEPARTING) Station ID: 2 | Level 14
-------------------------------------------------------------------------
(DESTINATION) Station ID: 2 | Level: 14
-------------------------------------------------------------------------
Minute | 2
(BIKE ARRIVAL) Station ID: 2 | Level: 14 | Bike ID: 0
-------------------------------------------------------------------------
Minute | 3
(CUSTOMER ARRIVAL) Station ID: 2 | Level: 14 | Capacity: 20
-------------------------------------------------------------------------
Minute | 4
(DEPARTING) Station ID: 1 | Level 7
-------------------------------------------------------------------------
(DESTINATION) Station ID: 1 | Level: 7
-------------------------------------------------------------------------
Minute | 5
(BIKE ARRIVAL) Station ID: 1 | Level: 7 | Bike ID: 0
---------------------------------